In [1]:
Pipeline_Id = 1            
Pipeline_Name = "LoadingDataFromSourceToSilver" 

In [2]:
jdbc_url = "jdbc:sqlserver://sqlserver8341.database.windows.net:1433;database=GoldLayer"
connection_properties = {
    "user": "suneetha",
    "password": "Suni@123",
    "driver": "com.microsoft.sqlserver.jdbc.SQLServerDriver"
}

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import regexp_replace,col,substring, substring_index, instr,concat_ws,md5,to_timestamp,lit,monotonically_increasing_id, lit,current_timestamp
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType,DateType,TimestampType
import re

source_schema = StructType([
                        StructField("id", IntegerType(), True),
                        StructField("batter", StringType(), True),
                        StructField("bowler", StringType(), True),
                        StructField("non_striker", StringType(), True),
                        StructField("runs_batter", IntegerType(), True),
                        StructField("runs_extras", IntegerType(), True),
                        StructField("runs_total", IntegerType(), True),
                        StructField("delivery_number", IntegerType(), True),
                        StructField("over_number", IntegerType(), True),  
                        StructField("team", StringType(), True),
                        StructField("inning_number", IntegerType(), True),
                        StructField("match_name", StringType(), True),
                        StructField("match_id", StringType(), True),
                        StructField("match_city", StringType(), True),
                        StructField("match_venue", StringType(), True),
                        StructField("match_type", StringType(), True),
                        StructField("team_type", StringType(), True),
                        StructField("match_start_date", DateType(), True),
                        StructField("player_out", StringType(), True),
                        StructField("_corrupt_record", StringType(), True)

                    ])


In [87]:
def process_files():
        mssparkutils.fs.mount(
            "abfss://bronzelayer@gen2storageforazurepoc.dfs.core.windows.net/",
            "/mountname",
            {"linkedService" : "synapsepoc123-WorkspaceDefaultStorage"}
        )

        files = mssparkutils.fs.ls(f'file:{mssparkutils.fs.getMountPath("/mountname")}')

        pattern = r"(\d{4}-\d{2}-\d{2}) (\d{2}:\d{2}:\d{2})"

        destination = spark.sql("SELECT COALESCE(MAX(date_time), CAST('1999-01-01' AS TIMESTAMP)) AS max_date_time FROM Cricket_Analysis_Data").collect()
        max_date_time = destination[0]["max_date_time"] if destination and destination[0]["max_date_time"] else None

        # looping each file in the directory 
        for file_info in files:
            file_path = file_info.path
            file_name = file_info.name      
            match = re.search(pattern, file_name)
            
            if match:
                date_time = f"{match.group(1)} {match.group(2)}"
            
                df = spark.createDataFrame([(file_name, date_time)], ["file_name", "date_time"])
                df = df.withColumn("date_time", to_timestamp("date_time", "yyyy-MM-dd HH:mm:ss"))

                if max_date_time:
                    if df.collect()[0]["date_time"] > max_date_time:
                        print(f"Source date_time ({df.collect()[0]['date_time']}) is newer. Proceed with processing {file_name}.")

                        path = f"abfss://bronzelayer@gen2storageforazurepoc.dfs.core.windows.net/{file_name}"
                        print(path)
                    
                        stage_df = spark.read.option("header","true").schema(source_schema).option("mode", "PERMISSIVE").option("columnNameOfCorruptRecord", "_corrupt_record").csv(path)
                        #display(stage_df)
                        
                        stage_df.cache()

                        cn_df = stage_df.where(col("_corrupt_record").isNotNull()).count()

                        if cn_df>0:

                            #Handling Corrupted Records...
                            print("Corrupted Records Found. Started loading into corruptedrecords container and also in process_corrupt_record table... ")

                            currupted_records= stage_df.where(col("_corrupt_record").isNotNull())
                            currupted_records.write.option("header",True).mode("append").csv("abfss://corruptedrecords@gen2storageforazurepoc.dfs.core.windows.net/corrupted records/")

                            currupted_records_df = stage_df \
                                .withColumn("Pipeline_Id", lit(Pipeline_Id)) \
                                .withColumn("Pipeline_Name", lit(Pipeline_Name)) \
                                .withColumn("file_name", lit(file_name)) \
                                .withColumn("path", lit(path)) \
                                .withColumn("date_time", to_timestamp(current_timestamp(), "yyyy-MM-dd HH:mm:ss")) \
                                .select("Pipeline_Id", "Pipeline_Name", "file_name", "path", "_corrupt_record", "date_time").where(col("_corrupt_record").isNotNull())

                            currupted_records_df.write.jdbc(url=jdbc_url, table="logs.process_corrupt_record", mode="append", properties=connection_properties)
                            print("Corrupted records loaded into a storage and processed_corrupt_record table successfully")
                        else:
                            print("No corrupted Records found")

                        # Processing Non corrupted records
                        print("Processing Non corrupted records.......")
                        source_df = stage_df.where(col("_corrupt_record").isNull()).drop("_corrupt_record")
                        display(source_df)

                        #extracting the date time from the file
                        file_path = f"{path}"
                        pattern = r"(\d{4}-\d{2}-\d{2}) (\d{2}:\d{2}:\d{2})"
                        match = re.search(pattern, file_path)

                        if match:
                            date_time = f"{match.group(1)} {match.group(2)}"
                            print(date_time)
                        
                        else:
                            print("No date and time found in the file name.")


                        # working on transformations
                        transforming_source_df = source_df.withColumn("batter", regexp_replace(col("batter"), "[^a-zA-Z0-9- ]", ""))\
                                            .withColumn("bowler", regexp_replace(col("bowler"), "[^a-zA-Z0-9- ]", ""))\
                                            .withColumn("non_striker", regexp_replace(col("non_striker"), "[^a-zA-Z0-9- ]", ""))\
                                            .withColumn("player_out", regexp_replace(col("player_out"), "-", ""))\
                                            .fillna({"runs_batter": 0, "runs_extras": 0,"runs_total": 0})\
                                            .withColumn("match_id", substring_index(col("match_id"), ".", 1))\
                                            .withColumn("date_time", to_timestamp(lit(date_time), "yyyy-MM-dd HH:mm:ss"))

                        #Generating hash key
                        df_with_hash = transforming_source_df.withColumn(
                            "hash_key", 
                            md5(
                                concat_ws(
                                    "_", 
                                    col("batter"),
                                    col("bowler"),
                                    col("non_striker"),
                                    col("runs_batter"),
                                    col("runs_extras"),
                                    col("runs_total"),
                                    col("delivery_number"),
                                    col("over_number"),
                                    col("inning_number"),
                                    col("match_name"),
                                    col("match_id"),
                                    col("match_city"),
                                    col("match_venue"),
                                    col("match_type"),
                                    col("team_type"),
                                    col("match_start_date"),
                                    col("player_out")
                                )
                            )
                        )
                        
                        #display(df_with_hash)
                        print("Hash code Generated")

                        df_with_hash.createOrReplaceGlobalTempView("df_with_hash_table")
                        print("Hash dataframe converted to temp table")

                        spark.sql(f"""MERGE INTO {dest_table_name} AS target
                        USING global_temp.df_with_hash_table AS source
                        ON target.hash_key = source.hash_key
                        WHEN NOT MATCHED THEN
                            INSERT (id, batter, bowler, non_striker, runs_batter, runs_extras, runs_total, delivery_number, over_number, team, inning_number, match_name, match_id, match_city, match_venue, match_type, team_type, match_start_date, player_out,date_time,hash_key)
                            VALUES (source.id, source.batter, source.bowler, source.non_striker, source.runs_batter, source.runs_extras, source.runs_total, source.delivery_number, source.over_number, source.team, source.inning_number, source.match_name, source.match_id, source.match_city, source.match_venue, source.match_type, source.team_type, source.match_start_date, source.player_out,source.date_time,source.hash_key);
                        """)
                        print(f"Inserted data into {dest_table_name}")
                         
                    else:
                        print("No need to process. Already processed Data")
                else:
                    print("No need to process")
            else:
                print(f"No date and time found in the file name: {file_name}")
            

In [89]:
dest_database_name = "default"
dest_schema_name = "dbo"
dest_table_name = "Cricket_Analysis_Data"

result = spark.sql(f"SHOW TABLES IN {dest_database_name}").filter(f"tableName = '{dest_table_name}'").count()

if result > 0:
    process_files()
else:
    schema = StructType([
                            StructField("id", IntegerType(), True),
                            StructField("batter", StringType(), True),
                            StructField("bowler", StringType(), True),
                            StructField("non_striker", StringType(), True),
                            StructField("runs_batter", IntegerType(), True),
                            StructField("runs_extras", IntegerType(), True),
                            StructField("runs_total", IntegerType(), True),
                            StructField("delivery_number", IntegerType(), True),
                            StructField("over_number", IntegerType(), True),  
                            StructField("team", StringType(), True),
                            StructField("inning_number", IntegerType(), True),
                            StructField("match_name", StringType(), True),
                            StructField("match_id", StringType(), True),
                            StructField("match_city", StringType(), True),
                            StructField("match_venue", StringType(), True),
                            StructField("match_type", StringType(), True),
                            StructField("team_type", StringType(), True),
                            StructField("match_start_date", DateType(), True),
                            StructField("player_out", StringType(), True),
                            StructField("date_time", TimestampType(), True),
                            StructField("hash_key", StringType(), True),
                        ])

    lake_df = spark.createDataFrame([], schema)
    lake_df.write.format("delta").mode("append").saveAsTable(f"{dest_table_name}")
    process_files()


In [4]:
jdbc_url = "jdbc:sqlserver://sqlserver8341.database.windows.net:1433;database=GoldLayer"
connection_properties = {
    "user": "suneetha",
    "password": "Suni@123",
    "driver": "com.microsoft.sqlserver.jdbc.SQLServerDriver"
}

In [14]:
Gold_Team_Read = spark.read.jdbc(url=jdbc_url, table="gold.gl_Team",  properties=connection_properties)
Gold_Team_Read.show()

<b>Team info</b>

In [24]:
from pyspark.sql import functions as F


# Load the data
df = spark.sql("""
    SELECT Team_Name, Team_Type 
        FROM (
        SELECT DISTINCT team AS Team_Name, team_type AS Team_Type, COUNT(*) AS con
        FROM cricket_analysis_data
        GROUP BY team, team_type
        ) AS teams_dataset
""")

Team_df_with_id = df.withColumn("Team_ID", F.abs(F.hash(F.col("Team_Name"), F.col("Team_Type"))))

Team_df = Team_df_with_id.select("Team_ID", "Team_Name", "Team_Type").distinct()

Gold_Team_Read = spark.read.jdbc(url=jdbc_url, table="[gold].[Team_Dim]",  properties=connection_properties)
Gold_Team_Read

Gold_Team_Join_df = Team_df.join(Gold_Team_Read, Team_df["Team_ID"] == Gold_Team_Read["Team_ID"],"left_anti")
Gold_Team_Join_df.show()

Gold_Team_Join_df.write.jdbc(url=jdbc_url, table="[gold].[Team_Dim]", mode="append", properties=connection_properties)
print("Loaded successfully")


<b>player info</b>

In [29]:
from pyspark.sql import functions as F

df = spark.sql("""
     SELECT DISTINCT player_name
        FROM (
            SELECT DISTINCT batter AS player_name FROM cricket_analysis_data
            UNION
            SELECT DISTINCT bowler AS player_name FROM cricket_analysis_data
        ) AS player_data
""")

player_df_with_id = df.withColumn("Player_ID", F.abs(F.hash(F.col("player_name"))))

players_df = player_df_with_id.select("Player_ID", "Player_Name").distinct()

#players_df.show()

Player_Gold_Read = spark.read.jdbc(url=jdbc_url, table="[gold].[Player_Dim]",  properties=connection_properties)
#Player_Gold_Read.show()

Player_Gold_Join_df = players_df.join(Player_Gold_Read, players_df["Player_ID"] == Player_Gold_Read["Player_ID"],"left_anti")
Player_Gold_Join_df.show()

Player_Gold_Join_df.write.jdbc(url=jdbc_url, table="[gold].[Player_Dim]", mode="append", properties=connection_properties)
print("Loaded successfully")


<b> relational table for players and team</b>

In [34]:
from pyspark.sql import functions as F

df = spark.sql("""
SELECT data.match_venue AS Match_Venue
	,data.match_city AS Match_City
FROM (
	SELECT DISTINCT match_venue
		,match_city
		,row_number() OVER (
			PARTITION BY match_venue ORDER BY match_city DESC
			) AS rownum
	FROM stage_match_details
    Where Match_Venue IS NOT NULL
	) AS data
WHERE rownum = 1
""")

Location_df_with_id = df.withColumn("Location_ID", F.abs(F.hash(F.col("match_venue"))))

Location_df = Location_df_with_id.select("Location_ID", "Match_Venue","Match_City").distinct()

Location_df.show()

# Reading data from the gold layer from location table

Location_Gold_Read = spark.read.jdbc(url=jdbc_url, table="[gold].[Location_Dim]",  properties=connection_properties)
Location_Gold_Read.show()

Location_Gold_Join_df = Location_df.join(Location_Gold_Read, Location_df["Location_ID"] == Location_Gold_Read["Location_ID"],"left_anti")
Location_Gold_Join_df.show()

Location_Gold_Join_df.write.jdbc(url=jdbc_url, table="[gold].[Location_Dim]", mode="append", properties=connection_properties)
print("Loaded successfully")




In [10]:
from pyspark.sql import functions as F

df = spark.sql("""
     select DISTINCT player_name,team  FROM (
    select distinct batter as player_name,team from cricket_analysis_data
    UNION
    select distinct bowler as player_name,team from cricket_analysis_data
    ) as player_data 
""")


player_join_df = df.join(Team_df, df.team == Team_df.Team_Name,"inner") \
                 .join(players_df1, df.player_name == players_df1.Player_Name,"inner")


player_team_df = player_join_df.select("Player_ID", "Team_ID").distinct()

player_team_df.write.format("delta").mode("append").saveAsTable("stage_Player_Team_Relation")


# player_team_df.write.jdbc(url=jdbc_url, table="Stage.stage_Player_Team_Relation", mode="overwrite", properties=connection_properties)

# print("Loaded successfully")


<b> Match Info</b>

In [37]:
from pyspark.sql import functions as F

df = spark.sql("""
SELECT ca.match_id as Match_Id
	,ca.match_name as Match_Name
	,match_type as Match_Type
	,Match_venue as Match_Venue
	,inning_number as Inning_Number
	,match_start_date as Match_Start_Date
FROM cricket_analysis_data ca
JOIN stage_Match_Location b ON ca.match_venue = b.match_venue

""")

# Match_df = df.select("Match_Id", "Match_Name","Match_Type","Inning_Number","Match_Venue","Match_Start_Date").orderBy("match_id")

# Match_df.show()

Join_match_location = df.join(Location_df, Match_df.Match_Venue == Location_df.Match_Venue,"inner")

# Match_df = df.select("Match_Id", "Match_Name","Match_Type","Inning_Number","Match_Venue","Match_Start_Date").orderBy("match_id")

Join_match_location.show()

#Match_df.write.format("delta").mode("append").saveAsTable("Fact_Match_Details")


# Match_df.write.jdbc(url=jdbc_url, table="Stage.stage_Match_Details", mode="overwrite", properties=connection_properties)

# print("Loaded successfully")


<b> Creating Fact Table </b>

In [13]:
from pyspark.sql.functions import *

# Create the initial DataFrame from a SQL query
events_df = spark.sql("""
SELECT batter          AS Batter,
       bowler          AS Bowler,
       non_striker     AS Non_Striker,
       runs_batter     AS Runs_Batter,
       runs_extras     AS Runs_Extras, 
       runs_total      AS Runs_Total,
       delivery_number AS Delivery_Number,
       over_number     AS Over_Number,
       inning_number   AS Inning_Number,
       team            AS Team,
       team_type       AS Team_Type,
       match_id        AS Match_Id,
       player_out      AS Player_Out,
       date_time       AS Silver_Load_date
FROM cricket_analysis_data
""")
print(events_df.count())


Team_join_df = events_df.join(Team_df,events_df.Team == Team_df.Team_Name, "left").distinct()
print(Team_join_df.count())

result_df = Team_join_df.drop("Team","Team_Type","Team_Name")

batter_join_df = (result_df.join(players_df1, result_df.Batter == players_df1.Player_Name, "left"))


player_join_df = batter_join_df.select(
    F.col("Player_ID").alias("Batter_Id"),  
    result_df["Batter"], 
    result_df["Bowler"], 
    result_df["Non_Striker"], 
    result_df["Runs_Batter"], 
    result_df["Runs_Extras"], 
    result_df["Runs_Total"], 
    result_df["Delivery_Number"], 
    result_df["Over_Number"], 
    result_df["Inning_Number"], 
    result_df["Team_ID"],
    result_df["Match_Id"], 
    result_df["Player_Out"], 
    result_df["Silver_Load_date"]
)

#display(player_join_df)
display(player_join_df.count())

Blower_join_df = (result_df.join(players_df1, result_df.Bowler == players_df1.Player_Name, "left"))

player_join_df1 = Blower_join_df.select(
    result_df["Batter"], 
    F.col("Player_ID").alias("Bowler_Id"),  
    result_df["Bowler"], 
    result_df["Non_Striker"], 
    result_df["Runs_Batter"], 
    result_df["Runs_Extras"], 
    result_df["Runs_Total"], 
    result_df["Delivery_Number"], 
    result_df["Over_Number"], 
    result_df["Inning_Number"], 
    result_df["Team_ID"],
    result_df["Match_Id"], 
    result_df["Player_Out"], 
    result_df["Silver_Load_date"]
)

display(player_join_df1.count())

two_pair = player_join_df1.join(player_join_df, player_join_df1.Batter == player_join_df.Batter, "inner").select(player_join_df["Batter_Id"],player_join_df1["*"])

# Select desired columns
#display(two_pair)

#section 3

non_stricket_join_df = (two_pair.join(players_df1, two_pair.Non_Striker == players_df1.Player_Name, "left"))

three_df = non_stricket_join_df.select(non_stricket_join_df["Player_ID"].alias("non_stricker_id"),two_pair["*"])

final_df1 = three_df.select(
    three_df["Batter_Id"], 
    three_df["Bowler_Id"], 
    three_df["non_stricker_id"], 
    three_df["Runs_Batter"], 
    three_df["Runs_Extras"], 
    three_df["Runs_Total"], 
    three_df["Delivery_Number"], 
    three_df["Over_Number"], 
    three_df["Inning_Number"], 
    three_df["Team_ID"],
    three_df["Match_Id"], 
    three_df["Player_Out"], 
    three_df["Silver_Load_date"]
).distinct()
print("done")
display(final_df1.count())

#section 4
Player_Out_join_df = (three_df.join(players_df1, three_df.Player_Out == players_df1.Player_Name, "left"))

Four_df = Player_Out_join_df.select(Player_Out_join_df["Player_ID"].alias("Player_Out_Id"),three_df["*"])

final_df2 = Four_df.select(
    Four_df["Batter_Id"], 
    Four_df["Bowler_Id"], 
    Four_df["non_stricker_id"], 
    Four_df["Runs_Batter"], 
    Four_df["Runs_Extras"], 
    Four_df["Runs_Total"], 
    Four_df["Delivery_Number"], 
    Four_df["Over_Number"], 
    Four_df["Inning_Number"], 
    Four_df["Team_ID"],
    Four_df["Match_Id"], 
    Four_df["Player_Out_Id"], 
    Four_df["Silver_Load_date"]
).distinct()
display(final_df2.count())

print("done section 4")

# final_df2.write.jdbc(url=jdbc_url, table="Stage.stage_Fact_Cricket_Details", mode="overwrite", properties=connection_properties)

# print("Loaded successfully")

final_df2.write.format("delta").mode("append").saveAsTable("stage_Main_Cricket_Details")




In [ ]:
select distinct a.Match_Id,a.Team_ID from 
cricket_analysis_data a
join stage.stage_Match_Details b
on a.Match_Id = b.Match_Id
join Stage.stage_Team c
on a.Team_ID = c.Team_ID